In [1]:
from gensim import corpora, models, similarities
import pandas as pd

reviews = pd.read_json('Reviews20160219.json')

In [2]:
reviews.shape

(27080, 5)

In [3]:
reviews.head()

,ReviewerExperience,date,name,rating,review
0,382,2012-12-10,Tian Tian Hainanese Chicken Rice,5,I don't even like 海南雞飯 based on past attempts....
1,44,2014-06-18,Tian Tian Hainanese Chicken Rice,5,"Pros: chicken rice is delicious Cons: Hot, no ..."
2,285,2013-06-18,Tian Tian Hainanese Chicken Rice,4,The rice is freakin incredible!!! Wow I was su...
3,112,2013-02-04,Tian Tian Hainanese Chicken Rice,2,DUDE! WHAT HAPPENED TO THIS PLACE?! I kept hea...
4,5,2014-05-31,Tian Tian Hainanese Chicken Rice,4,The Chicken Rice is Singapore's national dish ...


In [4]:
reviews.drop_duplicates().groupby('name').size().sort_values(ascending=False)

name
Din Tai Fung                                   222
Jumbo Seafood                                  162
Tian Tian Hainanese Chicken Rice               151
Wild Honey                                      94
Ippudo                                          88
Paradise Dynasty                                78
Song Fa Bak Kut Teh                             76
Tim Ho Wan                                      69
Saveur                                          68
PS.Cafe                                         65
Itacho Sushi                                    61
Tiong Bahru Bakery                              61
Nam Nam Noodle Bar                              60
Boon Tong Kee                                   60
Maison Ikkoku Cocktail Bar                      55
CÉ LA VI                                        53
Food For Thought                                52
Omakase Burger                                  48
Kith Cafe                                       47
Pizzeria Mozza            

In [5]:
filtered_reviews = reviews[(reviews.name != "Din Tai Fung")&\
        (reviews.name != "Tian Tian Hainanese Chicken Rice")&\
        (reviews.name != "Jumbo Seafood")]

In [6]:
rev_text_norm = pd.Series(filtered_reviews.drop_duplicates().review)
rev_text_norm.size

19799

In [7]:
import re, unicodedata
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

In [8]:
def remove_non_ascii(string):
    ### remove unreadable unicode characters
    return unicodedata.normalize('NFKD', string).encode('ascii','ignore')
    #return string.decode('unicode_escape').encode('ascii','ignore')

def review_to_words(raw_review):
    ascii_only = remove_non_ascii(raw_review)
    letters_only = re.sub("[^a-zA-Z]", " ", ascii_only)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    #return( " ".join( meaningful_words ))
    return meaningful_words

In [9]:
texts = [review_to_words(x) for x in rev_text_norm]

In [10]:
texts[0:5]

[['us',
  'would',
  'read',
  'papers',
  'mini',
  'chicken',
  'rice',
  'war',
  'maxwell',
  'really',
  'care',
  'less',
  'curiousity',
  'got',
  'better',
  'decided',
  'head',
  'check',
  'ah',
  'tai',
  'chicken',
  'rice',
  'presentation',
  'standard',
  'expected',
  'hawker',
  'stalls',
  'meat',
  'tender',
  'light',
  'soy',
  'dressing',
  'refreshing',
  'clincher',
  'rice',
  'whichi',
  'glistening',
  'chicken',
  'fat',
  'full',
  'flavours',
  'help',
  'even',
  'though',
  'total',
  'stalls',
  'selling',
  'chicken',
  'rice',
  'queues',
  'ah',
  'tai',
  'always',
  'long',
  'sure',
  'get',
  'early',
  'avoid',
  'disappointment',
  'closes',
  'everything',
  'sold'],
 ['ex',
  'chef',
  'tian',
  'tian',
  'brings',
  'us',
  'ah',
  'tai',
  'couple',
  'stalls',
  'famous',
  'tian',
  'tian',
  'stall',
  'maxwell',
  'food',
  'centre',
  'fan',
  'tian',
  'tian',
  'knew',
  'must',
  'try',
  'ah',
  'tai',
  'happy',
  'find',
  'ove

In [11]:
>>> # remove words that appear only once
>>> from collections import defaultdict
>>> frequency = defaultdict(int)
>>> for text in texts:
>>>     for token in text:
>>>         frequency[token] += 1

In [12]:
token

'couples'

In [13]:
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
#from pprint import pprint   # pretty-printer
#pprint(texts)

In [14]:
dictionary = corpora.Dictionary(texts)

In [15]:
print(dictionary)

Dictionary(22771 unique tokens: [u'gai', u'woods', u'clotted', u'hanging', u'woody']...)


In [16]:
print(dictionary.token2id)

{u'gai': 7582, u'woods': 2163, u'clotted': 2291, u'hanging': 2110, u'woody': 12427, u'trawling': 11434, u'localized': 16069, u'chatter': 8608, u'originality': 5830, u'refunding': 17449, u'fattiness': 12943, u'bringing': 3358, u'broiler': 16435, u'wooden': 6746, u'wholemeal': 2497, u'wednesday': 3767, u'broiled': 2524, u'capsicum': 9764, u'stereotypical': 16915, u'bbqs': 10287, u'sooth': 18633, u'scraped': 2186, u'errors': 8791, u'cooking': 1364, u'succumb': 13347, u'ching': 20192, u'china': 2143, u'wagyu': 3217, u'affiliated': 17175, u'doldrums': 19378, u'kids': 1796, u'uplifting': 13860, u'naturel': 15512, u'robata': 19951, u'climbed': 11872, u'preparazzi': 21346, u'controversy': 6145, u'spotty': 12345, u'golden': 848, u'stern': 12582, u'catchy': 14745, u'music': 4603, u'therefore': 6531, u'mitsuya': 20959, u'excl': 12158, u'caneles': 21722, u'circumstances': 14379, u'intake': 10967, u'morally': 17636, u'locked': 8778, u'busyness': 18194, u'coulda': 20550, u'wang': 8547, u'pints': 376

In [17]:
new_vec = dictionary.doc2bow(review_to_words(u"Chicken rice tastes so damn good so tastes"))
print(new_vec) 

[(9, 1), (26, 1), (59, 1), (245, 2), (511, 1)]


In [18]:
>>> corpus = [dictionary.doc2bow(text) for text in texts] # all sentences converted into bag of words
#>>> corpora.MmCorpus.serialize('deerwester.mm', corpus) # store to disk, for later use
#>>> print(corpus)

In [19]:
tfidf = models.TfidfModel(corpus)

In [20]:
corpus_tfidf = tfidf[corpus]
#for doc in corpus_tfidf:
#    print(doc)

In [21]:
>>> #### SERIALIZATION OF TRANSFORMATIONS to save memory space
>>> lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50) # initialize an LSI transformation
>>> corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
>>> # transformed our tf-ldf corpus via latent semantic indexing into 2-D space (num_topics = 2)
>>> lsi.print_topics(10)

[(0,
  u'0.129*"chicken" + 0.114*"food" + 0.112*"good" + 0.108*"place" + 0.107*"rice" + 0.100*"really" + 0.096*"like" + 0.095*"great" + 0.093*"one" + 0.091*"service"'),
 (1,
  u'-0.423*"ramen" + -0.258*"rice" + -0.256*"chicken" + -0.211*"soup" + -0.192*"noodles" + 0.189*"coffee" + 0.180*"pizza" + -0.146*"fried" + -0.139*"pork" + -0.119*"broth"'),
 (2,
  u'0.757*"ramen" + -0.294*"chicken" + -0.252*"rice" + -0.158*"thai" + -0.123*"fried" + -0.117*"curry" + 0.114*"broth" + -0.098*"fish" + 0.087*"ippudo" + -0.076*"crab"'),
 (3,
  u'-0.680*"sushi" + 0.222*"coffee" + -0.218*"sashimi" + -0.212*"japanese" + -0.129*"salmon" + -0.099*"quality" + 0.097*"pizza" + -0.090*"fish" + 0.088*"burger" + 0.084*"eggs"'),
 (4,
  u'0.650*"pizza" + -0.381*"coffee" + 0.132*"pizzas" + 0.130*"pasta" + -0.128*"breakfast" + -0.123*"thai" + 0.121*"crust" + 0.118*"crab" + -0.104*"cafe" + -0.101*"toast"'),
 (5,
  u'0.358*"thai" + 0.358*"pizza" + -0.234*"crab" + 0.200*"chicken" + 0.144*"curry" + -0.142*"burger" + 0.133

In [22]:
#### SERIALIZATION OF TRANSFORMATIONS to save memory space
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=50) # initialize an LSI transformation

lda_model.print_topics(10)

[(17,
  u'0.013*dumpling + 0.008*wan + 0.008*tim + 0.007*maki + 0.007*liberty + 0.006*szechuan + 0.005*laptop + 0.005*hostess + 0.005*requests + 0.005*therapy'),
 (10,
  u'0.014*glutinous + 0.012*sheng + 0.010*kampong + 0.009*shisha + 0.008*chestnut + 0.007*mackerel + 0.007*shortcake + 0.006*ny + 0.006*tradition + 0.006*macaron'),
 (49,
  u'0.009*chatterbox + 0.008*blueberry + 0.008*lattes + 0.008*ikan + 0.007*bilis + 0.006*chopsticks + 0.005*touches + 0.005*americano + 0.005*breko + 0.004*smiles'),
 (13,
  u'0.012*centres + 0.012*tei + 0.011*payoh + 0.011*toa + 0.010*hangout + 0.008*spending + 0.008*imperial + 0.008*btw + 0.007*heated + 0.007*reuben'),
 (28,
  u'0.005*udon + 0.004*mocha + 0.003*japanese + 0.003*vintage + 0.003*stars + 0.003*sashimi + 0.003*meet + 0.003*japan + 0.003*tempura + 0.003*industrial'),
 (39,
  u'0.011*quiche + 0.011*scones + 0.009*ginseng + 0.009*scone + 0.008*hardware + 0.007*church + 0.006*hospital + 0.006*indie + 0.006*soooo + 0.006*huat'),
 (7,
  u'0.013

In [23]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
46     43.096928        1       1 -0.340155 -0.059130
41     15.164264        1       2 -0.323939 -0.063201
18      9.817040        1       3 -0.308647  0.019652
40      9.179997        1       4 -0.301972  0.008003
29      3.120509        1       5 -0.212596  0.011308
28      2.709207        1       6 -0.202385 -0.026937
25      1.604012        1       7 -0.175696 -0.024836
33      1.021952        1       8 -0.113258 -0.081792
6       1.021933        1       9 -0.065408  0.051690
0       1.013144        1      10 -0.067355  0.248369
5       0.909371        1      11 -0.112661  0.147894
35      0.663928        1      12 -0.076003 -0.038553
48      0.650201        1      13 -0.054643  0.007442
23      0.544092        1      14 -0.012497  0.089324
17      0.484888        1      15 -0.010802 -0.016057
21      0.479100        1      16 -0.006842 -0.017130
3       0.474898        1      17  0.014402 -0.030192
2       0.471636        1      18 -0.027750 -0.017056
22      0.454561        1      19 -0.002294  0.004998
12      0.398071        1      20  0.053251 -0.013852
16      0.371159        1      21  0.004797  0.007013
49      0.353779        1      22 -0.012040 -0.007638
45      0.347533        1      23  0.031524  0.028716
15      0.323588        1      24  0.022222 -0.011443
34      0.302032        1      25  0.047642 -0.039817
24      0.281070        1      26  0.054460 -0.036147
39      0.280205        1      27  0.078251  0.000989
9       0.268133        1      28  0.101967 -0.002140
7       0.255542        1      29  0.035386 -0.028663
20      0.248260        1      30  0.055656 -0.011432
1       0.239232        1      31  0.059402 -0.011586
37      0.228671        1      32  0.090455 -0.013493
26      0.227306        1      33  0.057533 -0.005621
36      0.222194        1      34  0.072482 -0.017237
38      0.220916        1      35  0.085314  0.008624
27      0.219517        1      36  0.106449 -0.010705
44      0.216714        1      37  0.077963 -0.008814
8       0.206450        1      38  0.099393  0.006999
13      0.192708        1      39  0.074766 -0.017908
32      0.188150        1      40  0.096698 -0.009808
10      0.182885        1      41  0.101546 -0.002219
19      0.182281        1      42  0.109870 -0.000939
31      0.174817        1      43  0.113438  0.003018
47      0.172658        1      44  0.087986 -0.011824
14      0.166572        1      45  0.097060 -0.004131
11      0.153535        1      46  0.117344 -0.003133
30      0.150876        1      47  0.110678  0.005911
42      0.122028        1      48  0.121823 -0.001355
4       0.116439        1      49  0.121455 -0.000989
43      0.105019        1      50  0.125730 -0.004172, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
1478   Default  4078.000000      coffee  4078.000000  30.0000  30.0000
2759   Default  1339.000000         mee  1339.000000  29.0000  29.0000
3474   Default   798.000000       ramen   798.000000  28.0000  28.0000
17553  Default   975.000000       latte   975.000000  27.0000  27.0000
10512  Default  1573.000000   chocolate  1573.000000  26.0000  26.0000
18718  Default  1173.000000       toast  1173.000000  25.0000  25.0000
20232  Default   770.000000         dim   770.000000  24.0000  24.0000
22351  Default  1449.000000        eggs  1449.000000  23.0000  23.0000
9586   Default  1356.000000         fun  1356.000000  22.0000  22.0000
18218  Default  3539.000000       great  3539.000000  21.0000  21.0000
4908   Default   750.000000         sum   750.000000  20.0000  20.0000
12974  Default  1387.000000       pizza  1387.000000  19.0000  19.0000
15912  Default  2481.000000     noodles  2481.000000  18.0000  18.0000
7895   Default  4273.000000        food  4273.000000  17.0000  17.0000
116    Defaul